In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import csv
import time

# Import API key
from config import api_key

In [2]:
# Webscrapping to Grab Zip Code Data
url = "https://worldpopulationreview.com/zips"
tables = pd.read_html(url)
table_1 = tables[0]

# Use Panada Series to Get Back Lost Zeros
table_1["Zip Code"] = table_1['Zip Code'].astype(str)
width = 5
table_1['Zip Code'] = table_1['Zip Code'].str.zfill(width)
table_1

,Zip Code,City,County,Population
0,77449,Katy,Harris County,128294
1,77494,Katy,Fort Bend County,118291
2,11368,Corona,Queens County,112088
3,79936,El Paso,El Paso County,111620
4,90011,Los Angeles,Los Angeles County,111165
...,...,...,...,...
495,33458,Jupiter,Palm Beach County,58714
496,22192,Woodbridge,Prince William County,58690
497,60638,Chicago,Cook County,58669
498,00976,Trujillo Alto,"Trujillo Alto County, PR",58640


In [4]:
# Create an array of the data
zips_array = pd.array(table_1['Zip Code'])

# Select the 150 Random Cities
rand_cities = np.random.choice(a=zips_array, replace = False, size=150)
rand_cities

array(['20019', '11435', '93536', '78577', '93312', '28027', '11372',
       '92804', '92376', '94538', '60016', '33157', '93274', '90042',
       '74012', '78641', '30044', '77449', '11432', '77494', '10452',
       '85326', '92801', '77083', '92507', '78521', '75227', '72401',
       '87114', '89108', '78254', '85122', '60628', '90066', '47906',
       '21234', '27610', '60634', '78613', '27858', '11691', '11385',
       '78245', '76179', '20011', '92630', '60632', '28078', '90001',
       '76137', '60657', '94806', '33411', '23452', '76028', '30906',
       '95624', '63376', '75040', '29445', '91950', '93065', '77379',
       '85225', '90250', '95355', '92592', '95076', '10457', '42101',
       '11375', '79912', '27587', '95035', '33142', '10032', '92114',
       '10312', '93535', '93230', '90006', '33125', '11210', '00926',
       '92703', '33511', '95630', '19143', '93306', '94587', '11229',
       '30043', '98682', '92563', '11746', '94122', '11234', '06010',
       '92115', '330

In [5]:
# Create Zip DF
zip_df = pd.DataFrame({
    "Zip Code": rand_cities
})

# Merge with Inner to get info for Rand Zips
merged_zip_df = pd.merge(zip_df, table_1, on = "Zip Code", how = "inner")


# Check for Duplicates
check = merged_zip_df.loc[merged_zip_df.duplicated(subset=["Zip Code"])]
check

# Write to a CSV 
merged_zip_df.to_csv("Resources/rand_zip_df.csv", index = False, header = True)

In [6]:
# Read in Housing Data
house_data_df = pd.read_csv("Resources/RDC_Inventory_Core_Metrics_Zip.csv")

# Use Panada Series to Get Back Lost Zeros
house_data_df["postal_code"] = house_data_df["postal_code"].astype(str)
width = 5
house_data_df["postal_code"] = house_data_df["postal_code"].str.zfill(width)

house_data_df = house_data_df.rename(columns = {"postal_code" : "Zip Code"})
# Create CSV for Rand Zips
merged_house_zip = pd.merge(house_data_df, zip_df, on = "Zip Code", how = "inner")

# Clean up df
cleaned_merged_house_zip = merged_house_zip.loc[:, ["Zip Code", "median_listing_price", "active_listing_count"]]
cleaned_merged_house_zip = cleaned_merged_house_zip.rename(columns = {"median_listing_price" : "Median Listing Price", "active_listing_count" : "Active Listing Count"})

# Write to CSV
cleaned_merged_house_zip.to_csv("Resources/home_prices_df.csv", index = False, header = True)

C:\Users\artsy\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [51]:
# Read in rand_csv
rand_cities_df = pd.read_csv("Resources/rand_zip_df.csv")
# rand_cities_short = rand_cities.iloc[0:5, 0:]
# rand_cities_array = rand_cities_short["Zip Code"].astype(str)
# rand_cities_short = rand_cities_short.loc[:, ["Zip Code"]]
# rand_cities_array
rand_cities_df = rand_cities["Zip Code"].astype(str)
# rand_zip_df = rand_cities_df.loc[:, ["Zip Code"]]
rand_cities_df

0      20019
1      11435
2      93536
3      78577
4      93312
       ...  
145    28173
146    30127
147     7047
148    85041
149    33025
Name: Zip Code, Length: 150, dtype: object

In [54]:
# Get JSON from API with Zipcode List
url = "https://realty-mole-property-api.p.rapidapi.com/zipCodes/"

headers = {
    'x-rapidapi-host': "realty-mole-property-api.p.rapidapi.com",
    'x-rapidapi-key': api_key
    }

data = []


for zips in rand_cities_df:
    
    time.sleep(5)
    
    zip_url = url + zips
    

    try:
        
        response = requests.request("GET", zip_url, headers=headers)
    
        resp = response.json()
        
        zipCode = resp["id"]
        rentals = resp["rentalData"]["totalRentals"]
        avgprice = resp["rentalData"]["averageRent"]
        
        
        data.append({
            "zipcode": zipCode,
            "avgprice": avgprice,
            "totalrentals": rentals
        })

        
        
    except:
        print("Zip Not Found")
        pass


Zip Not Found
Zip Not Found
Zip Not Found
Zip Not Found


In [57]:
# Create Data Frame
rental_df = pd.DataFrame(data)

# Export to Json File
rental_df.to_json("Resources/rental_data.json")

In [58]:
# Read in Json File
rent_df = pd.read_json(r"Resources/rental_data.json")

# Export to CSV File
rent_df.to_csv(r"Resources/rental_data.csv", index = None)

In [59]:
rent_check = pd.read_csv("Resources/rental_data.csv")
rent_check

,zipcode,avgprice,totalrentals
0,20019,1785.60,7
1,11435,2799.21,178
2,93536,2375.31,80
3,78577,1477.23,118
4,93312,1992.15,81
...,...,...,...
141,33027,3113.45,440
142,28173,2152.45,143
143,30127,1772.60,222
144,85041,1754.91,218


AttributeError: 'list' object has no attribute 'to_json'